# Lab #4, part II: cover inequalities

In part I we saw how to add inequalities to tighten a formulation. We now look at another well-known problem in Optimization, the Knapsack problem. Given a set $N$ of $n$ objects, each with a value $v_i$ and a weight $p_i$, find the subset of $N$ that maximizes the total value while maintaining the total weight below a given limit $q$.

We can model the problem as follows:
$$
\begin{array}{lll}
\max & \sum_{i\in N} v_i x_i\\
\textrm{s.t.} & \sum_{i\in N} p_i x_i \le q\\
              & x_i \in \{0,1\} & \forall i\in N
\end{array}
$$

As for part I, we consider the _continuous_ (i.e. LP) relaxation of the problem, where $x_i \in \{0,1\}$ is replaced by $0 \le x_i \le 1$.

Let's define the data first.

In [ ]:
# When using Colab, make sure you run this instruction beforehand
!pip install --upgrade cffi==1.15.0
import importlib
import cffi
importlib.reload(cffi)
!pip install mip

In [ ]:
import numpy as np
import mip

np.random.seed(987654320)

n = 10  # number of items

N = range(n)

q = 210  # capacity of the knapsack
v = np.random.randint(100, size=n)  # value of each item
p = np.random.randint(100, size=n)  # weight of each item

In [ ]:
# The continuous relaxation of the knapsack problem

m = mip.Model()

# Create a vector of n continuous variables with values between 0 and 1 TODO

# Set the objective function TODO

# Write the constraint TODO

m.optimize()

print("Values: ", v)
print("Weights:", p)
print("Capacity:", q)
print(f"objective: {m.objective_value:.3f}; solution:", [x[i].x for i in N])

The solution has only one fractional variable. This is not casual: the solution to the continuous relaxation of _any_ knapsack problem, with the sole knapsack constraint, has at most one fractional variable (why?).

In order to improve the dual bound, similar to the location problem in part I we can add constraints: the resulting restriction will have a _smaller_ dual bound, i.e. a smaller upper bound (because this is a maximization problem).

## Cover inequalities

Given the set $N=0,1,2,\ldots{},n-1$, a vector $p = (p_0,p_1,\ldots{},p_{n-1})$ of $n$ elements and a scalar $q$, a _cover_ $C$ is a subset of $N$ such that

$$
\sum_{i\in C} p_i > q.
$$

In other words, a cover is a selection of objects that exceeds the knapsack's capacity. Therefore, an optimal solution of the knapsack problem _cannot_ contain all items in the cover, but at least one of them must be excluded. We can translate this to a constraint: for any cover $C$, the following inequality is valid:

$$
\sum_{i\in C} x_i \le |C| - 1.
$$

A cover for our problem is $\{0, 2, 3\}$, because $p_0 + p_2 + p_3 = 29 + 96 + 95 = 220 > q$. These three items canot be all picked together, so we can easily add the constraint

$$
x_0 + x_2 + x_3 \le 2.
$$

In [ ]:
# Add the newly cover inequality to the problem TODO
m.add_constr()
m.optimize()
print(f"New objective: {m.objective_value:.3f}")
print("Solution:", [x[i].x for i in N])


In [ ]:
print(p[0] + p[4] + p[5] + p[7] + p[8] + p[9])


This constraint is useless though, because the current solution does _not_ violate it: $1 + 0 + 0 = 1 \le 2$. In fact the objective function does not change---and neither does the solution. We need a cover that yields a _violated_ cover inequality: for example, the subset of all nonzero variables $\{0, 4, 5, 7, 8, 9\}$:

$$
x_0 + x_4 + x_5 + x_7 + x_8 + x_9 \le 5.
$$

In [ ]:
# Add the cover inequality TODO
m.add_constr()
m.optimize()
print(f"New objective: {m.objective_value:.3f}")
print("Solution:", [x[i].x for i in N])

In [ ]:
print(p[0] + p[3] + p[5] + p[7] + p[8] + p[9])
print(x[0].x + x[3].x + x[5].x + x[7].x + x[8].x + x[9].x)


Now we're talking! The objective function decreased (as expected) and the solution changed, but there is still a fractional value. The values close to 1 but not exactly 1 are simply due to numerical issues, we can treat them as integer. Let's try again with $C=\{0, 3, 5, 7, 8, 9\}$:

$$
x_0 + x_3 + x_5 + x_7 + x_8 + x_9 \le 5
$$

In [ ]:
# Add the cover inequality TODO
m.add_constr(x[0] + x[3] + x[5] + x[7] + x[8] + x[9] <= 5)

m.optimize()
print(f"New objective: {m.objective_value:.3f}")
print([x[i].x for i in N])

Better still, but the solution now is a lot more fractional.

How do we automate this? We need a method that
* Receives as an input $p$, $q$, and the current solution vector $x^*$;
* Finds a cover $C$ such that $\sum_{i\in C} p_i > q$ (otherwise it is not a cover) and $\sum_{i\in C} x^*_i > |C| - 1$ (otherwise it is not violated);
* Add the cover inequality and re-optimizes
* Repeat from the start with an updated $x^*$.

Ideally we could find a $C$ that _maximizes_ $\sum_{i\in C} x^*_i - (|C| - 1)$: if the result is positive, then we found a violated inequality, otherwise there is no _violated_ cover inequality.

Finding a cover $C$ can be modeled as an ILP! Let's call it the _separation_ ILP and model it:

* sets: $N$;
* parameters: $x^*_i$ for $i \in N$, vector $p$ and scalar $q$;
* variables: $y_i$ for $i\in N$: it is 1 if we include $i$ in the cover, 0 otherwise.

With these variables, $|C|$ is defined by $\sum_{i\in N} y_i$. The expression $\sum_{i\in C} x^*_i - (|C| - 1)$, which is our objective function (to be maximized), is

$$
\sum_{i\in N} x^*_i y_i - (\sum_{i\in N} y_i - 1) = \sum_{i\in N} (x^*_i - 1) y_i + 1.
$$ 

Is it linear?

The only constraint is given by the cover condition, i.e., $\sum_{i\in C} p_i > q$. We can't enforce $>$ constraints but, since all $p_i$ are integer, $y_i$ are binary, and $q$ is integer, we replace $> q$ with $\ge q+1$.

$$
\sum_{i\in N} p_i y_i \ge q + 1
$$


In [ ]:
def separate(N, xstar, p, q):
    
    sepm = mip.Model()  # Create a new problem! This will have binary variables
    
    # Add binary variables called y TODO
    y = [sepm.add_var(var_type=mip.BINARY) for i in N]
    
    sepm.objective = mip.maximize(mip.xsum((xstar[i] - 1) * y[i] for i in N) + 1)
    # set the objective function for the separation problem TODO
    
    sepm.add_constr(mip.xsum(p[i] * y[i] for i in N) >= q + 1) # what is the constraint we need to add? TODO
    sepm.optimize()
    if sepm.objective_value <= 1e-5:  # Objective function is nonpositive ==> no violated cover inequality exists
        return None
    return [i for i in N if y[i].x > 0.5]  # check y > 0.5 for the aforementioned numerical issues

# Cutting plane algorithm:
# loop until no new inequalities are found

while True:
    cover = separate(N, [x[i].x for i in N], p, q)

    if cover is None:
        break

    print("Cover: ", cover)
    # If the cover is not None, add the cover inequality TODO
    m.add_constr(mip.xsum(x[i] for i in cover) <= len(cover) - 1)
    m.optimize()
    print(f"New objective: {m.objective_value:.3f}")
    print("Solution:", end='')
    for i in N:
        print(f"{x[i].x:5.3f} ", end='')
    print("")

    

In five iterations we managed to improve the dual bound from 367 to 358.

Let's solve the original problem to optimality: we create a knapsack problem where all variables are binary.

In [ ]:
m2 = mip.Model()

xb = [m2.add_var(var_type=mip.BINARY) for _ in N]  # New variables, binary this time

m2.objective = mip.maximize(mip.xsum(v[i] * xb[i] for i in N)) # set the objective function TODO

m2.add_constr(mip.xsum(p[i] * xb[i] for i in N) <= q)  # Add the constraint TODO

m2.optimize()

print(f"objective: {m2.objective_value:.3f}; solution:", [xb[i].x for i in N])